<a href="https://colab.research.google.com/github/jpantojaj/Credit_Scoring_Specialization/blob/main/Sesi%C3%B3n_16_Seguimiento_de_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Seguimiento de Nuevas Cosechas**

### **1. Carga Inicial de Librerías**

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline

### **2. Carga y Análisis inicial de datos**

In [ ]:
df_nc = pd.read_csv('Base_SolicitudesCreditoEfectivo_NuevasCosechas.csv', sep = ",")
df_nc.head()

In [ ]:
df_ref = pd.read_csv('Base_SolicitudesCreditoEfectivo_Test.csv', sep = ",", index_col=False)
df_ref.head()

#### Corregimos los tipos de variables, este paso es circunstancial

In [ ]:
df_ref['CODMES']=df_ref['CODMES'].astype(str)
df_ref['CODSOLICITUD']=df_ref['CODSOLICITUD'].astype(str)
df_ref['MIN_MES_DE_DEFAULT']=df_ref['MIN_MES_DE_DEFAULT'].astype(str)
df_ref['FLG_GARANTIA']=df_ref['FLG_GARANTIA'].astype(str)
df_ref['TARJETA_RELACIONADA']=df_ref['TARJETA_RELACIONADA'].astype(str)
df_ref['VEHICULAR_RELACIONADA']=df_ref['VEHICULAR_RELACIONADA'].astype(str)
df_ref['HIPOTECARIO_RELACIONADA']=df_ref['HIPOTECARIO_RELACIONADA'].astype(str)
df_ref['CLASIF_SISTEMA_ULT_12M']=df_ref['CLASIF_SISTEMA_ULT_12M'].astype(str)
df_ref['FLG_PDH']=df_ref['FLG_PDH'].astype(str)
df_ref['FLG_TC_VISA']=df_ref['FLG_TC_VISA'].astype(str)
df_ref['FLG_TC_MC']=df_ref['FLG_TC_MC'].astype(str)

In [ ]:
df_nc['CODMES']=df_nc['CODMES'].astype(str)
df_nc['CODSOLICITUD']=df_nc['CODSOLICITUD'].astype(str)
df_nc['MIN_MES_DE_DEFAULT']=df_nc['MIN_MES_DE_DEFAULT'].astype(str)
df_nc['FLG_GARANTIA']=df_nc['FLG_GARANTIA'].astype(str)
df_nc['TARJETA_RELACIONADA']=df_nc['TARJETA_RELACIONADA'].astype(str)
df_nc['VEHICULAR_RELACIONADA']=df_nc['VEHICULAR_RELACIONADA'].astype(str)
df_nc['HIPOTECARIO_RELACIONADA']=df_nc['HIPOTECARIO_RELACIONADA'].astype(str)
df_nc['CLASIF_SISTEMA_ULT_12M']=df_nc['CLASIF_SISTEMA_ULT_12M'].astype(str)
df_nc['FLG_PDH']=df_nc['FLG_PDH'].astype(str)
df_nc['FLG_TC_VISA']=df_nc['FLG_TC_VISA'].astype(str)
df_nc['FLG_TC_MC']=df_nc['FLG_TC_MC'].astype(str)

#### Como se ve nuestra muestra de referencia

In [ ]:
sns.countplot(data = df_ref, x = "FLG_DEFAULT_12M")
target_count = df_ref.FLG_DEFAULT_12M.value_counts()
print('# Buen_Pagador:', target_count[0])
print('# 1 Mora_12M:', target_count[1])
print('Bad rate:', target_count[1]/(target_count[0]+target_count[1]))

In [ ]:
a1=df_ref.pivot_table(values="CODSOLICITUD", index="CODMES", aggfunc="count", sort=True)
a1.plot(kind = 'bar',
       #stacked = 'True',          # Muestra las barras apiladas
       alpha = 0.4,               # nivel de transparencia
       width = 0.9,               # Grosor de las barras para dejar espacio entre ellas
       figsize=(9,4));            # Cambiamos el tamaño de la figura

In [ ]:
a2=df_ref.pivot_table(values="FLG_DEFAULT_12M", index="CODMES", aggfunc="mean", sort=True)
a2.plot(alpha = 0.4, figsize=(9,4), ylim=(0.04,0.09))

####  Como se ve la nueva muestra

In [ ]:
sns.countplot(data = df_nc, x = "FLG_DEFAULT_12M")
target_count = df_nc.FLG_DEFAULT_12M.value_counts()
print('# Buen_Pagador:', target_count[0])
print('# 1 Mora_12M:', target_count[1])
print('Bad rate:', target_count[1]/(target_count[0]+target_count[1]))

In [ ]:
a1=df_nc.pivot_table(values="CODSOLICITUD", index="CODMES", aggfunc="count", sort=True)
a1.plot(kind = 'bar',
       #stacked = 'True',          # Muestra las barras apiladas
       alpha = 0.4,               # nivel de transparencia
       width = 0.9,               # Grosor de las barras para dejar espacio entre ellas
       figsize=(9,4));            # Cambiamos el tamaño de la figura

In [ ]:
a2=df_nc.pivot_table(values="FLG_DEFAULT_12M", index="CODMES", aggfunc="mean", sort=True)
a2.plot(alpha = 0.4, figsize=(9,4), ylim=(0.04,0.09))

### **3. Carga de los artefactos desarrollados en el entranamiento**

#### En esta parte, llamamos a los pickles del Feature Engineering como del Modelo Final para su uso

In [ ]:
import pickle

In [ ]:
pip install feature_engine

In [ ]:
# Cargar el pipeline del feature engineering
with open('fe_pipeline.pickle','rb') as fe_data_file:
     fe_final = pickle.load(fe_data_file)

In [ ]:
# Cargar el modelo
with open('final_model.pickle','rb') as modelFile:
     modelo_final = pickle.load(modelFile)

### Declaremos las variables que se usaran de las Bases o MDTs cargadas

In [ ]:
cat_cols_2=['SEGMENTOCLIENTE','CLASIF_SISTEMA_ULT_12M','FLG_PDH','PROFESION','ZONA_DEL_DESEMBOLSO','ESTADO_CIVIL']

In [ ]:
num_cols_2=['PLAZO_CREDITO','MESES_AHORROS_ULT_6M','MEDIANA_AHORROS_ULT_6M','NUMERO_DE_PAGOS_PDH', 'INGRESO_CLIENTE',
            'EDAD_T','LINEA_DE_TC', 'MONTO_TC_MEMBRESIA']

### Con lo anterior apliquemos los pickles a nuestras muestras

In [ ]:
#Probemos el pipeline
df_ref_xt=fe_final.transform(df_ref.drop(['FLG_DEFAULT_12M'],axis=1))
df_nc_xt=fe_final.transform(df_nc.drop(['FLG_DEFAULT_12M'],axis=1))
df_ref_y=df_ref['FLG_DEFAULT_12M']
df_nc_y=df_nc['FLG_DEFAULT_12M']

In [ ]:
df_ref_xt_sel=pd.concat([df_ref_xt[cat_cols_2],df_ref_xt[num_cols_2]],axis=1)
df_nc_xt_sel=pd.concat([df_nc_xt[cat_cols_2],df_nc_xt[num_cols_2]],axis=1)

### **4. ROC y GINI**

#### Primero, evaluemos el poder predictivo del modelo en la nueva muestra, comparado con las métricas obtenidas en la muestra de referencia (test)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
# Probemos el modelo
pred_ref = modelo_final.predict_proba(df_ref_xt_sel)
pred_nc = modelo_final.predict_proba(df_nc_xt_sel)
print('BaseLine roc-auc: ',roc_auc_score(df_ref_y, pred_ref[:,1]), 'GINI: ', 2*roc_auc_score(df_ref_y, pred_ref[:,1])-1 )
print('New roc-auc: ', roc_auc_score(df_nc_y, pred_nc[:,1]), 'GINI: ', 2*roc_auc_score(df_nc_y, pred_nc[:,1])-1 )

#### Podemos complementar este análisis con un Bootstrapping de la muestra nueva, para ver si el GINI inicial se encuentra dentro de los posibles valores generados

In [ ]:
from sklearn.utils import resample

In [ ]:
bootstrap_iter = 1000
roc_auc = []

In [ ]:
for i in range(bootstrap_iter):
    X_, y_ = resample(df_nc_xt_sel, df_nc_y)
    y_pred = modelo_final.predict_proba(X_)
    acc = roc_auc_score(y_,y_pred[:,1])
    roc_auc.append(acc)

In [ ]:
roc_auc_final = np.array(roc_auc)

In [ ]:
gini_final=2*roc_auc_final-1

In [ ]:
print('GINI')
print('Average: ', gini_final.mean())
print('Standard deviation: ', gini_final.std())

In [ ]:
#bandas de confianza
from scipy.stats import norm
z_value = norm.ppf((1 + 0.99) / 2)

In [ ]:
LS=gini_final.mean()+z_value*gini_final.std()
LS

In [ ]:
LI=gini_final.mean()-z_value*gini_final.std()
LI

In [ ]:
sns.kdeplot(gini_final, shade=True)
plt.title('Distribución del GINI')
plt.xlabel('GINI')
plt.ylabel('Densidad')
plt.show()

#### Luego, también es importante verificar como se ve el poder discriminatorio de cada variable en la nueva muestra vs el baseline (referencia)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
roc_values_1 = []
for feature in df_ref_xt_sel.columns:
    clf = DecisionTreeClassifier(max_depth= 4, min_samples_split=0.4674032789842478, random_state=0)
    clf.fit(df_ref_xt_sel[feature].to_frame(), df_ref_y)
    y_scored = clf.predict_proba(df_ref_xt_sel[feature].to_frame())
    roc_values_1.append(roc_auc_score(df_ref_y, y_scored[:, 1]))

GINI_Base = pd.Series(roc_values_1)*2-1
GINI_Base.index = df_ref_xt_sel.columns
GINI_Base.sort_values(ascending=False).plot.bar(figsize=(20, 5))
plt.ylabel('GINI Baseline')

In [ ]:
roc_values_2 = []
for feature in df_nc_xt_sel.columns:
    clf = DecisionTreeClassifier(max_depth= 4, min_samples_split=0.4674032789842478, random_state=0)
    clf.fit(df_nc_xt_sel[feature].to_frame(), df_nc_y)
    y_scored = clf.predict_proba(df_nc_xt_sel[feature].to_frame())
    roc_values_2.append(roc_auc_score(df_nc_y, y_scored[:, 1]))

GINI_New = pd.Series(roc_values_2)*2-1
GINI_New.index = df_nc_xt_sel.columns
GINI_New.sort_values(ascending=False).plot.bar(figsize=(20, 5))
plt.ylabel('GINI New')

In [ ]:
comparativa=pd.concat([pd.DataFrame(GINI_Base,columns=['GINI_Base']), pd.DataFrame(GINI_New,columns=['GINI_New'])], axis=1)
comparativa['Diferencia']=comparativa['GINI_New']-comparativa['GINI_Base']
comparativa

### **5. PSI y CSI**

#### EL PSI (Population Stability Index) busca medir la estabilidad de la población basado en la comparación de la distribución por rangos fijos o percentilicos, a partir de un baseline o muestra de referencia

In [ ]:
pd.DataFrame(pred_ref[:,1]).describe()

In [ ]:
def scale_range (input, min, max):
    input += -(np.min(input))
    input /= np.max(input) / (max - min)
    input += min
    return input

In [ ]:
buckets = 10
raw_breakpoints = np.arange(0, buckets + 1) / (buckets) * 100
breakpoints = scale_range(raw_breakpoints, np.min(pred_ref[:,1]), np.max(pred_ref[:,1]))

In [ ]:
initial_counts = np.histogram(pred_ref[:,1], breakpoints)[0]
new_counts = np.histogram(pred_nc[:,1], breakpoints)[0]

In [ ]:
df = pd.DataFrame({'Bucket': np.arange(1, 11), 'Breakpoint Value':breakpoints[1:], 'Initial Count':initial_counts, 'New Count':new_counts})
df['Initial Percent'] = df['Initial Count'] / len(pred_ref[:,1])
df['New Percent'] = df['New Count'] / len(pred_nc[:,1])

In [ ]:
df['New Percent'][df['New Percent'] == 0] = 0.001

In [ ]:
df

In [ ]:
percents = df[['Initial Percent', 'New Percent', 'Bucket']] \
             .melt(id_vars=['Bucket']) \
             .rename(columns={'variable':'Population', 'value':'Percent'})

In [ ]:
percents

In [ ]:
p = sns.barplot(x="Bucket", y="Percent", hue="Population", data=percents)
p.set(xlabel='Bucket', ylabel='Population Percent')
sns.despine(left=True)

In [ ]:
df['PSI'] = (df['New Percent'] - df['Initial Percent']) * np.log(df['New Percent'] / df['Initial Percent'])

In [ ]:
df

In [ ]:
np.sum(df['PSI'])

#### Según este valor, no observamos una variación significativa en las poblaciones de referencia y nueva

#### Seteamos una función para calcular el PSI y CSI

In [ ]:
def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):
    def psi(expected_array, actual_array, buckets):
        def scale_range (input, min, max):
            input += -(np.min(input))
            input /= np.max(input) / (max - min)
            input += min
            return input

        breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

        if buckettype == 'bins':
            breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
        elif buckettype == 'quantiles':
            breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])

        expected_fractions = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
        actual_fractions = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

        def sub_psi(e_perc, a_perc):
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return(value)

        psi_value = sum(sub_psi(expected_fractions[i], actual_fractions[i]) for i in range(0, len(expected_fractions)))

        return(psi_value)

    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[1 - axis])

    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = psi(expected, actual, buckets)
        elif axis == 0:
            psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
        elif axis == 1:
            psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

    return(psi_values)

In [ ]:
calculate_psi(pred_ref[:,1], pred_nc[:,1], buckettype='bins', buckets=10, axis=0)

In [ ]:
calculate_psi(pred_ref[:,1], pred_nc[:,1], buckettype='quantiles', buckets=10, axis=0)

#### EL CSI (Characteristic Stability Index) es similar al PSI pero sobre las distribuciones de cada variable

In [ ]:
# Rangos Fijos
print("CSI - Rangos Fijos")
for col in df_ref_xt_sel.columns:
    csi_values = calculate_psi(df_ref_xt_sel[col].values, df_nc_xt_sel[col].values, buckettype='bins', buckets=10, axis=0)
    csi = np.sum(csi_values)
    print(f'{col} -> {csi=:.4f}')

In [ ]:
# Rangos Percentílicos
print("CSI - Rango Percentílicos")
for col in df_ref_xt_sel.columns:
    csi_values = calculate_psi(df_ref_xt_sel[col].values, df_nc_xt_sel[col].values, buckettype='quantile', buckets=10, axis=0)
    csi = np.sum(csi_values)
    print(f'{col} -> {csi=:.4f}')

### **6. Calibración**

#### Veamos rápidamente la PD Promedio vs la RD de la nueva muestra

In [ ]:
print('La PD Promedio Baseline es: ', pred_ref[:,1].mean(), 'El RD Baseline es de: ', df_ref_y.mean())

In [ ]:
print('La PD Promedio Nueva es: ', pred_nc[:,1].mean(), 'El RD Nueva es de: ', df_nc_y.mean())

In [ ]:
from sklearn.calibration import calibration_curve
# Una función para evaluar la calibración
def plot_calibration_curve(y_true, probs, bins, strategy):

    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_true, probs, n_bins=bins, strategy=strategy)

    max_val = max(mean_predicted_value)

    plt.figure(figsize=(8,10))
    plt.subplot(2, 1, 1)
    plt.plot(mean_predicted_value, fraction_of_positives, label='PD vs RD')
    plt.plot(np.linspace(0, max_val, bins), np.linspace(0, max_val, bins),
         linestyle='--', color='red', label='Perfect calibration')

    plt.xlabel('Probability Predictions')
    plt.ylabel('Fraction of positive examples')
    plt.title('Calibration Curve')
    plt.legend(loc='upper left')


    plt.subplot(2, 1, 2)
    plt.hist(probs, range=(0, 1), bins=bins, density=True, stacked=True, alpha=0.3)
    plt.xlabel('Probability Predictions')
    plt.ylabel('Fraction of examples')
    plt.title('Density')
    plt.show()

#### Como se ve en la muestra inicial o de test

In [ ]:
plot_calibration_curve(df_ref_y, pred_ref[:, 1], bins=10, strategy='uniform')

#### Como se ve en la nueva cosecha

In [ ]:
plot_calibration_curve(df_nc_y, pred_nc[:, 1], bins=10, strategy='uniform')

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
# Calibración Sigmoide
cal_sigmoid = CalibratedClassifierCV(modelo_final, cv='prefit', method='sigmoid')
cal_sigmoid.fit(df_nc_xt_sel, df_nc_y)
prob_sigmoid = cal_sigmoid.predict_proba(df_nc_xt_sel)[:, 1]

In [ ]:
plot_calibration_curve(df_nc_y, prob_sigmoid, bins=10, strategy='uniform')

In [ ]:
prob_sigmoid

In [ ]:
print('La PD Promedio Nueva es: ', prob_sigmoid.mean(), 'El RD Nueva es de: ', df_nc_y.mean())